In [1]:
import argparse
import numpy as np
import torch.nn as nn
import torch.utils.data
from torchsummary import summary
from utils.dataset import load_mat_hsi, sample_gt, HSIDataset
from utils.utils import split_info_print, metrics, show_results
from utils.scheduler import load_scheduler
from models.get_model import get_model
from train import train, test
from utils.utils import Draw
import time
from sklearn.utils import shuffle
from tqdm import tqdm
import torch.backends.cudnn as cudnn
import os
import random

In [2]:
parser = argparse.ArgumentParser(description="run patch-based HSI classification")
parser.add_argument("--model", type=str, default='gscvit') # model name
parser.add_argument("--dataset_name", type=str, default="UP") # dataset name
parser.add_argument("--dataset_dir", type=str, default="../data") # dataset dir
parser.add_argument("--device", type=str, default="0")
parser.add_argument("--patch_size", type=int, default=8) # patch_size
parser.add_argument("--num_run", type=int, default=5)
parser.add_argument("--epoch", type=int, default=200)
parser.add_argument("--bs", type=int, default=128)  # bs = batch size
parser.add_argument("--ratio", type=float, default=0.02) # ratio of training + validation sample

opts,_ = parser.parse_known_args()

device = torch.device("cuda:{}".format(opts.device))

# print parameters
print("experiments will run on GPU device {}".format(opts.device))
print("model = {}".format(opts.model))
print("dataset = {}".format(opts.dataset_name))
print("dataset folder = {}".format(opts.dataset_dir))
print("patch size = {}".format(opts.patch_size))
print("batch size = {}".format(opts.bs))
print("total epoch = {}".format(opts.epoch))
print("{} for training, {} for validation and {} testing".format(opts.ratio / 2, opts.ratio / 2, 1 - opts.ratio))

# load data
image, gt, labels = load_mat_hsi(opts.dataset_name, opts.dataset_dir)

num_classes = len(labels)

num_bands = image.shape[-1]

# random seeds
seeds = [202401, 202402, 202403, 202404, 202405, 202406, 202407, 202408, 202409, 202410]

# empty list to storing results
results = []


experiments will run on GPU device 0
model = gscvit
dataset = UP
dataset folder = ../data
patch size = 8
batch size = 128
total epoch = 200
0.01 for training, 0.01 for validation and 0.98 testing


In [3]:
def setup_seed(seed):
    # PyTorch CPU随机种子
    torch.manual_seed(seed)
    # 所有GPU的随机种子
    torch.cuda.manual_seed_all(seed)
    # Python哈希种子（影响字典等数据结构的行为）
    os.environ['PYTHONHASHSEED'] = str(seed)
    # NumPy随机种子
    np.random.seed(seed)
    # Python内置随机种子
    random.seed(seed)
    # 启用确定性算法（降低性能但保证可重复）
    torch.backends.cudnn.deterministic = True
    # 关闭自动优化（固定卷积算法选择）
    torch.backends.cudnn.benchmark = False

In [4]:
def setup_dataloaders(image, train_gt, val_gt, opts, seed):
    """
    设置数据加载器
    """
    # 创建数据集
    train_set = HSIDataset(image, train_gt, patch_size=opts.patch_size, data_aug=True)
    
    val_set = HSIDataset(image, val_gt, patch_size=opts.patch_size, data_aug=True)

    # 创建数据加载器
    generator = torch.Generator()
    generator.manual_seed(seed)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        batch_size=opts.bs,
        shuffle=True,
        drop_last=False,
        generator=generator,
        worker_init_fn=lambda worker_id: np.random.seed(seed + worker_id)
    )

    val_loader = torch.utils.data.DataLoader(
        dataset=val_set,
        batch_size=opts.bs,
        shuffle=False,
        drop_last=False
    )

    return train_loader, val_loader

In [5]:
time_all = []
for run in range(1):
    setup_seed(seeds[run])
    print("running an experiment with the {} model".format(opts.model))
    print("run {} / {}".format(run + 1, opts.num_run))

    # get train_gt, val_gt and test_gt
    train_gt, val_gt, test_gt = sample_gt(
            gt, 
            train_samples_per_class=30,  # 每类30个训练样本
            val_samples_per_class=10,    # 每类10个验证样本
            seed=seeds[run]
        )

# 设置数据加载器
    train_loader, val_loader = setup_dataloaders(
        image=image,
        train_gt=train_gt,
        val_gt=val_gt,
        opts=opts,
        seed=seeds[run]
    )

    # load model and loss
    model = get_model(opts.model, opts.dataset_name, opts.patch_size)
    model = model.to(device)
    if run == 0:
        split_info_print(train_gt, val_gt, test_gt, labels)
        print("network information:")
        with torch.no_grad():
            input_shape = (1, num_bands, opts.patch_size, opts.patch_size)
            summary(model, input_shape)

    model = model.to(device)
    # print(model)
    optimizer, scheduler = load_scheduler(opts.model, model)

    criterion = nn.CrossEntropyLoss()

    # where to save checkpoint model
    model_dir = "./checkpoints/" + opts.model + '/' + opts.dataset_name + '/' + str(run)

    try:
        train(model, optimizer, criterion, train_loader, val_loader, opts.epoch, model_dir, device, scheduler)
    except KeyboardInterrupt:
        print('"ctrl+c" is pused, the training is over')
    start = time.time()
    # test the model
    probabilities = test(model, model_dir, image, opts.patch_size, num_classes, device)

    prediction = np.argmax(probabilities, axis=-1)
    end = time.time()
    test_time = end - start
    time_all.append(test_time)
    # computing metrics
    run_results = metrics(prediction, test_gt, n_classes=num_classes)  # only for test set
    results.append(run_results)
    show_results(run_results, label_values=labels)

    # draw the classification map
    # Draw(model,image,gt,opts.patch_size,opts.dataset_name,opts.model,num_classes)
if opts.num_run > 1:
        show_results(results, label_values=labels, agregated=True)

running an experiment with the gscvit model
run 1 / 5
目标: 每类30个训练样本, 10个验证样本
------------------------------------------------------------
类别 0: 训练集 30样本, 验证集 10样本, 测试集 6591样本
类别 1: 训练集 30样本, 验证集 10样本, 测试集 18609样本
类别 2: 训练集 30样本, 验证集 10样本, 测试集 2059样本
类别 3: 训练集 30样本, 验证集 10样本, 测试集 3024样本
类别 4: 训练集 30样本, 验证集 10样本, 测试集 1305样本
类别 5: 训练集 30样本, 验证集 10样本, 测试集 4989样本
类别 6: 训练集 30样本, 验证集 10样本, 测试集 1290样本
类别 7: 训练集 30样本, 验证集 10样本, 测试集 3642样本
类别 8: 训练集 30样本, 验证集 10样本, 测试集 907样本

验证采样结果:
------------------------------------------------------------
类别                               训练集        验证集        测试集
------------------------------------------------------------
name1                             30         10       6591
name2                             30         10      18609
name3                             30         10       2059
name4                             30         10       3024
name5                             30         10       1305
name6                             30        

  0%|▌                                                                                                                           | 1/200 [00:00<01:34,  2.11it/s]

train at epoch 1/200, loss=2.428562
epoch = 1: best OA = 0.1111, loss = 2.428562
epoch = 1: current loss = 2.428562 (best loss = 2.428562)
epoch = 2: best OA = 0.1111, loss = 1.725430
epoch = 2: current loss = 1.725430 (best loss = 1.725430)


  2%|███                                                                                                                         | 5/200 [00:00<00:26,  7.23it/s]

epoch = 3: best OA = 0.1889, loss = 1.446032
epoch = 3: current loss = 1.446032 (best loss = 1.446032)
epoch = 4: current loss = 1.213050 (best loss = 1.213050)
train at epoch 5/200, loss=1.129685
epoch = 5: current loss = 1.129685 (best loss = 1.129685)


...

train at epoch 195/200, loss=0.162822


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:19<00:00, 10.50it/s]


train at epoch 200/200, loss=0.056381


inference on the HSI: 3256it [00:13, 244.42it/s]                                                                                                                 

Confusion matrix :
[[ 6168     0    14     0     0     0   353    56     0]
 [    0 16478     0   254     0  1877     0     0     0]
 [   17     0  1938     0     0     0     0   103     1]
 [   32    19     0  2944     0     7     0    14     8]
 [    1     0     0     0  1304     0     0     0     0]
 [    0   477     0     0     0  4512     0     0     0]
 [    5     0     0     0     0     0  1284     1     0]
 [   26     0   129    23     0     0     0  3464     0]
 [    0     0     0     0     5     0     0     0   902]]---
Accuracy : 91.93%
---
class acc :
	name1: 93.58
	name2: 88.55
	name3: 94.12
	name4: 97.35
	name5: 99.92
	name6: 90.44
	name7: 99.53
	name8: 95.11
	name9: 99.45
---
AA: 95.34%
Kappa: 89.47

Agregated results :
Confusion matrix :
[[6.1680e+03 0.0000e+00 1.4000e+01 0.0000e+00 0.0000e+00 0.0000e+00
  3.5300e+02 5.6000e+01 0.0000e+00]
 [0.0000e+00 1.6478e+04 0.0000e+00 2.5400e+02 0.0000e+00 1.8770e+03
  0.0000e+00 0.0000e+00 0.0000e+00]
 [1.7000e+01 0.0000e+00 1.93